In [7]:
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import IPython
import string
import math
import scipy.signal as sgn

import pyaudio
import wave

from helpers import *

# CONSTANTS

Fs = 16000 # sampling freqency
T=.6 # transmission time per symbol
TWOPI = 2 * np.pi

min_freq = 1000
max_freq = 2000

tolerance = 8 # the max ratio between the loudest and the less loud freqencies recieved

#----------------------------------

# Record initial noise !!

CHUNKSIZE = int(2*Fs) # 2 time slots

# initialize portaudio
p = pyaudio.PyAudio()
stream = p.open(format=pyaudio.paInt16, channels=1, rate=Fs, input=True, frames_per_buffer=CHUNKSIZE)

# do this as long as you want fresh samples
data1 = stream.read(CHUNKSIZE)
numpydata = np.fromstring(data1, dtype=np.int16)

noise = numpydata/1 # need to change the amplitude ??

# close stream
stream.stop_stream()
stream.close()
p.terminate()

# Check presence of silence between 1kHz and 2kHz
noise12 = butter_bandpass_filter(noise, 1200, 1800, Fs, order=10)
amplitude12 = np.mean(abs(noise12))

# Check presence of silence between 2kHz and 3kHz
noise23 = butter_bandpass_filter(noise, 2200, 2800, Fs, order=10)
amplitude23 = np.mean(abs(noise23))

NOISE_RANGE=0 
if (abs(amplitude12) < abs(amplitude23)):
    NOISE_RANGE=1
else:
    NOISE_RANGE=2

if(NOISE_RANGE==2):
    freq_shift=1000
else:
    freq_shift=0
    
my_alphabet = ['ü'] + list(string.printable) #our alphabet

#transforms a character from our alphabet to its binary representation
def toBinary(c):
    my_index = my_alphabet.index(c)
    toBinary = bin(my_index)
    return toBinary[2:] # we remove the header "0b"

number_bits = int(math.ceil(np.log2(len(my_alphabet)))) # the number of bits we require to represent our alphabet
my_base = np.linspace(min_freq, max_freq, num = math.ceil(number_bits+2)) # +2 because we dont need the borders (1000 and 2000)
#ON DOIT BIEN AUSSI SHIFTER LE LIN SPACE DANS CE CAS NON ??

base_distance = my_base[1] - my_base[0] # distance between 2 waves freqs

#adds to x a wave of frequency f
def wave(f,n):
    x = np.sin(TWOPI * (float(f + freq_shift)/Fs) * n) # If freq_shift=1000, all sines are shifted to the 1kHz-2kHz range.
    return x

#transforms a character from our alphabet to a distinct wave
def toWave(c):
    my_bin = toBinary(c)
    x = np.zeros(int(T * Fs))
    n = np.arange(len(x))
    for i in range(len(my_bin) -1, -1, -1):
        if(int(my_bin[i])):
            f = my_base[len(my_bin) - 1] # because of the binary base
            x += wave(f, n)
    return x

def toWave2(c) :
    my_bin = toBinary(c)
    raised = np.zeros(int(T * Fs))
    for i in range(len(my_bin) -1, -1, -1):
        if(int(my_bin[i])):    
            f = my_base[len(my_bin) - i] # because of the binary base
            raised += raised_cosine(f + freq_shift, T, Fs, roll_off=0.5)
    return raised

# open the message file
my_text = open('message.txt', 'r')
my_message = my_text.read()
my_text.close

# array of our waves
my_message_waves = []
for i in range(len(my_message)):
    # my_message_waves.append(toWave(my_message[i]))
    my_message_waves.append(toWave2(my_message[i]))
    
# Generate inital clap !!!
clap = np.random.normal(0, 3, size=2000)
z = np.zeros(Fs-2000)
## INITIAL_SILENCE = np.zeros(4*Fs)
clap = np.concatenate(( clap, z))

# we generate the wave to be sent
to_be_sent = []
to_be_sent = np.concatenate((to_be_sent, clap)) # add initial clap

for i in range(0, len(my_message_waves)):
    to_be_sent = np.concatenate((to_be_sent, my_message_waves[i]))
    
print(NOISE_RANGE)
IPython.display.Audio(to_be_sent, rate=Fs)


2
